## <i>Dog Breed Classifier</i>

<div class="alert alert-block alert-info">
<b>Tip:</b> Navigate to either jpg or png format image
    by clicking "Select" button then clicking in the directory 
    pane followed by scrolling and clicking to find 
    an image file of interest. Click the "Select" button for 
    the image file of choice. Click the "Classify" button and 
    wait for an assessment.  Repeat for another file by clicking 
    "Change" button, navigate to another file, then clicking 
    "Change" button again to update filename selected, then click 
    the "Classify" button.
</div>

In [ ]:
import numpy as np
from glob import glob
import os
import os
import cv2               
import matplotlib.pyplot as plt                        
%matplotlib inline
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image                  
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import GlobalAveragePooling2D
from keras.layers import Dense
from keras.models import Sequential
from extract_bottleneck_features import *
import pickle
from ipyfilechooser import FileChooser
import ipywidgets as widgets
from IPython.display import display
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# load list of dog names
infile = open('dognames.pkl','rb')
dog_names = pickle.load(infile)

# extract pre-trained face detector
face_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_alt.xml')

# define ResNet50 model
ResNet50_model = ResNet50(weights='imagenet')

### Obtain bottleneck features from another pre-trained CNN.
bottleneck_features = np.load('bottleneck_features/DogResnet50Data.npz')

### Define your architecture.
RS50_infer = Sequential()
RS50_infer.add(GlobalAveragePooling2D(input_shape=(1,1,2048)))
RS50_infer.add(Dense(133, activation='softmax'))
RS50_infer.load_weights('saved_models/weights.best.Resnet50.hdf5')

In [ ]:
def face_detector2(img_path):
    '''
    input image path
    convert path to image object
    convert object to grayscale
    classify based upon refined hyperparameters
    return boolean result as containing face or not
    '''
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.15,
        minNeighbors=5,
        minSize=(30, 30),
    )
    return len(faces) > 0

def path_to_tensor(img_path):
    '''
    input image path
    load image
    render into compatible tensor
    return tensor
    '''
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224, 224),
                         interpolation = "bicubic")
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    '''
    input list of image paths
    render paths to tensors
    return vertical stack of tensors
    '''
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

def ResNet50_predict_labels(img_path):
    '''
    input image path
    convert path to tensor
    preprocess the tensor
    make predictions
    return most probable prediction
    '''
    # returns prediction vector for image located at img_path
    img = preprocess_input(path_to_tensor(img_path))
    return np.argmax(ResNet50_model.predict(img))

def dog_detector(img_path):
    '''
    input image path
    retrieve most probable prediction
    classify prediction as likely a dog
    return boolean result
    '''
    prediction = ResNet50_predict_labels(img_path)
    return ((prediction <= 268) & (prediction >= 151))

def RS50_infer_predict(img_path):
    '''
    input image path
    convert image to tensor
    retrieve output of pre-trained network, aka bottleneck
    make prediction from full connected network
    return prediction of dog breed
    '''
    # extract bottleneck features
    bottleneck_feature = extract_Resnet50(path_to_tensor(img_path))
    # obtain predicted vector
    predicted_vector = RS50_infer.predict(bottleneck_feature)
    # return dog breed that is predicted by the model
    return dog_names[np.argmax(predicted_vector)]

def pic_display(path):
    '''
    load image path
    convert to RGB
    display in RGB
    '''
    img = cv2.imread(path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.axis('off')
    plt.show()

def infer_sort(path):
    '''
    import image path
    display image
    sort image by dog or human
    if dog, figure out the breed
    print sort decision
    '''
    # display
    pic_display(path)
    # sort class and reveal
    dog = dog_detector(path)
    human = face_detector2(path)
    if dog:
        print('A dog detected!')
        prob_breed = RS50_infer_predict(path)
        per_loc = prob_breed.find('.')+1
        print('Possible breed: '+ prob_breed[per_loc:])
    elif human:
        print('A human face detected!')
        prob_breed = RS50_infer_predict(path)
        per_loc = prob_breed.find('.')+1
        print('May share resemblance with: '+ prob_breed[per_loc:])
    else:
        print('Neither a dog nor a human face was detected.')
        
    print('----------------------------------------------')
    print()

In [ ]:
# Create and display a FileChooser widget
fc = FileChooser('/Users/crahan/FC demo')
# Set multiple file filter patterns (uses https://docs.python.org/3/library/fnmatch.html)
fc.filter_pattern = ['*.jpg', '*.png']
display(fc)

In [ ]:
button = widgets.Button(description="Classify!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    if fc.selected != None:
        with output:
            output.clear_output()
            infer_sort(fc.selected)

button.on_click(on_button_clicked)